In [34]:
from src.unet7 import Unet
from src.utils.config import read_json_config
from src.utils.io import image_mask_generator_from_npz
from pathlib import Path
from tensorflow.keras.preprocessing.image import \
    ImageDataGenerator as defaultImageDataGenerator
from tensorflow.keras.optimizers import Adam
from src.utils.losses import dice_bce_loss, dice_coeff
import tensorflow as tf
import numpy as np
from tqdm.auto import tqdm


In [2]:
CONFIG_PATH = '../configs/training_config.json'
CHECKPOINT_PATH = '../model/interesting_checkpoints/model.epoch349.hdf5'

In [3]:
config = read_json_config(CONFIG_PATH)
parameters = config['parameters']

In [15]:
model = Unet(parameters['target_height'], parameters['target_width'], filters=64, nclasses=1, do=0.0)
model.load_weights(CHECKPOINT_PATH)
model.compile(optimizer=Adam(learning_rate=parameters['start_lr']),
                loss=dice_bce_loss, metrics=[dice_coeff])

In [37]:
img_mask_gen_args_val = config['validation']['img_mask_gen_args']

image_mask_datagen_val = defaultImageDataGenerator(
    **img_mask_gen_args_val)
val_data_path = Path('..') / config['validation']['data_path']
image_mask_generator_val = image_mask_generator_from_npz(
    str(val_data_path),
    image_mask_datagen_val,
    config['validation']['batch_size'],
    parameters['seed'],
)

(3030, 256, 256, 1)
(3030, 256, 256, 1)


In [40]:
def evaluate(model, generator):
    dce = []
    loss = []
    t = 0
    for img, mask in tqdm(generator):
        pred = model.predict(img)
        dce.append(dice_coeff(pred, mask))
        loss.append(dice_bce_loss(pred, mask))
        t += 1
        if t > 2:
            break
    return np.mean(dce), np.mean(loss)

In [41]:
evaluate(model, image_mask_generator_val)

  0%|          | 0/61 [00:00<?, ?it/s]

(0.54722565, 0.55134237)